In [2]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib
from matplotlib.axes import Axes
%matplotlib inline

import numpy as np
import pyvista as pv
import pandas as pd
import xarray as xr
import time

import vtk
from vtk import *

import pyvista as pv
from pyvista import examples
pv.global_theme.jupyter_backend = 'pythreejs'

In [3]:
ds = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_31/output_WAOM_check/ocean_flt.nc')
dt = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_31/output_WAOM_check/ocean_avg_0001.nc')
#dd = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_31/output_WAOM_check/ocean_avg_0001.nc')
dg = xr.open_dataset('/scratch/project_2000789/boeiradi/waom10_frc/waom10extend_grd.nc')

### Plotting zice

In [ ]:
#plt.figure(figsize=(10, 10))
# masking land points
mask_rho = dt.variables['mask_rho']
zice = dt.variables['zice']

proj = ccrs.SouthPolarStereo(central_longitude=0.0, true_scale_latitude=None, globe=None)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1, 1, 1, projection=proj)
#plt.scatter(zpts[:, 0], zpts[:, 1], c=zpts[:, 2])
# another way
plt.pcolormesh(dg.lon_rho,dg.lat_rho, zice*mask_rho,transform=ccrs.PlateCarree(), vmin=-500, vmax=0)
plt.axis("image")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(zpts[:, 0], zpts[:, 1], c=zpts[:, 2])
plt.axis("image")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.colorbar()
plt.show()

In [ ]:
# zice points
def get_zice_points(zice):
    x = np.array([])
    y = np.array([])
    z = np.array([])
    for i in range(len(zice)):
        for j in range(len(zice[i])):
            if zice[i][j] != 0:
                x = np.append(x,j)
                y = np.append(y,i)
                z = np.append(z,zice[i][j])
                
    return x,y,z
test = dt.variables['zice'].values
xz,yz,zz = get_zice_points(test)

In [ ]:
point_cloud = pv.PolyData(zpts)
point_cloud

In [ ]:
# pv.global_theme.jupyter_backend = 'pythreejs'
pv.global_theme.jupyter_backend = 'ipyvtklink'
point_cloud.plot(eye_dome_lighting=True)

### Plotting using pyvista

In [4]:
x = (ds.variables['Xgrid'].values)
y = (ds.variables['Ygrid'].values)
z = (ds.variables['Zgrid'].values)

In [4]:
print(x.shape)

(87601, 15438)


In [5]:
def make_points(i):
    """Helper to make XYZ points"""
    z1 = list(z[:,i])
    x1 = list(x[:,i])
    y1 = list(y[:,i])
    return np.column_stack((x1, y1, z1))

In [6]:
points = make_points(5)

In [7]:
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly


line = lines_from_points(points)
line

PolyData,Information
N Cells,87600
N Points,87601
X Bounds,"4.191e+02, 4.304e+02"
Y Bounds,"9.690e+02, 1.010e+03"
Z Bounds,"3.075e+01, 3.084e+01"
N Arrays,0


In [1]:
def cast_all_points(points):
    start_time = time.time()
    network = pv.MultiBlock()
    network.add_field_data(np.arange(x.shape[0]/900),"time")
    #print(network)
    for i in range(0,2573):
        if (i%25 ==0):
            print(i,'/ 2573','in',(time.time()-start_time)/60)
        points = make_points(i)
        line = lines_from_points(points)
        line["days since start"] = np.arange(line.n_points)
        tube = line.tube(radius=0.1)
        network.append(tube)
        #line['time'] = np.arange(x.shape[0])
        
        #tube.plot(smooth_shading=True)
        
    # create the scalar bars and plot
    p = pv.Plotter()
    
    sargs = dict(
        n_labels=0,
        label_font_size = 14
    )
    annotations = {
        2820: '30', 5640: '60', 8460:'90',11280: '120', 14100:'150', 16920:'180',
        19740:'210', 22560:'240',
        25380:'270',28200:'300',31020:'330',33840:'360',36660:'390',
        39480:'420',42300:'450',45120:'480',47940:'510',50760:'540',
        53580:'570',56400:'600',59220:'630',62040:'660',64860:'690',
        67680:'720',70500:'750',73320:'780',76140:'810',78960:'840',
        81780:'870',84600:'900',87420:'930',
        
        
    }
    p.add_mesh(network, smooth_shading=True,annotations=annotations, scalar_bar_args=sargs)
    
    #_ = p.add_scalar_bar('Time', annotations=annotations, scalar_bar_args=sargs)
    
    #p.add_mesh(mesh, scalars='Elevation', annotations=annotations)
    
    _ = p.add_axes(line_width=5, labels_off=False)
    p.show()
    print('total time:',(time.time()-start_time)/60,'min')
cast_all_points(points)

NameError: name 'points' is not defined

In [9]:
# def make_points(i):
#     """Helper to make XYZ points"""
#     z1 = list(z[:,i])
#     x1 = list(x[:,i])
#     y1 = list(y[:,i])
#     return np.column_stack((x1, y1, z1))
# points = make_points(1)
# line = lines_from_points(points)

In [10]:
# line["scalars"] = np.arange(line.n_points)
# tube = line.tube(radius=0.1)
# tube.plot(smooth_shading=True)

In [11]:
# p = pv.Plotter()
# p.add_mesh(stream.tube(radius=0.0015))
# p.view_xy()
# p.show(cpos=cpos)